In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Property_Sales.csv')
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

recency : sale_date
frequency : 
monetization : sale_price

In [ ]:
# Get data for 2020
df_2020 = df[(df['sale_date']>='2020/01/01') & (df['sale_date']<='2020/12/31')].copy()
# Add column sale_datetime -> convert column sale_date to datetime type
df_2020['sale_datetime'] = pd.to_datetime(df_2020['sale_date'].str[:10], format='%Y/%m/%d')
# 2011/02/03 00:00:00+00

In [ ]:
df_2020.head()

In [ ]:
print('Min:{}; Max:{}'.format(min(df_2020.sale_datetime),
                            max(df_2020.sale_datetime)))

In [ ]:
import datetime as dt
snapshot_date = max(df_2020.sale_datetime) + dt.timedelta(days=1)
print(snapshot_date)

In [ ]:
# aggregate data on address level
# datamart = df_2020.groupby(['address']).count()
datamart = df_2020.groupby(['address']).agg({
    'sale_datetime': lambda x: (snapshot_date - x.max()).days,
    'sale_price' : 'sum',
    'object_id' : 'count'
})
datamart.rename(columns = {
    'sale_datetime' : 'recency',
    'object_id' : 'frequency',
    'sale_price' : 'monetary_value'
}, inplace=True)
datamart.head()

In [ ]:
r_labels = range(4,0,-1)
r_quartiles = pd.qcut(datamart['recency'],4,labels=r_labels)
datamart = datamart.assign(r=r_quartiles)
datamart.head()

In [ ]:
m_quartiles = pd.qcut(datamart['monetary_value'],4,labels=range(1,5))
datamart = datamart.assign(m=m_quartiles)
datamart.head()

In [ ]:
f_quartiles = pd.qcut(datamart['frequency'],4,labels=range(1,5))
# datamart = datamart.assign(f=f_quartiles)
# datamart.head()

In [ ]:
def join_rfm(x) : return str(x['r']) + str(x['m'])
datamart['segment'] = datamart.apply(join_rfm, axis=1)
datamart['score'] = datamart[['r','m']].sum(axis=1)
datamart.head()

In [ ]:
# Check Biggest Size / count of each segment
datamart.groupby('segment').size().sort_values(ascending=False)[:10]

In [ ]:
datamart[datamart['segment']=='11'][:5]

In [ ]:
datamart.groupby('score').agg({
    'recency' : 'mean',
    'monetary_value' : ['mean','count']
}).round(1)

In [ ]:
def segment_cat(df):
    if df['score']>= 7:
        return 'Premium'
    elif (df['score']>=4):
        return 'Normal'
    else :
        return 'Bad'
    
datamart['segment_category']=datamart.apply(segment_cat,axis=1)
datamart.groupby('segment_category').agg({
    'recency' : 'mean',
    'monetary_value' : ['mean','count']
}).round(1)